# Validation of single datasets

In this notebook we will explore how to validate an individual dataset. We will follow the next steps:

- Basic Exploratory Data Analysis
- Profile data & identify anomalies with Great Expectations
- Fixing problems detected using pandas
- Assesing data quality improvements after remediation steps were taken

To do so, we will use the datasets available in the `data` folder. These datasets are:

- `data/a.csv`: Data about traffic accidents from city of Toronto. Website: 
- `data/b.csv`: Data about traffic accidents from city of Toronto. Website:
- `data/c.csv`: Data about traffic accidents from city of Toronto. Website:



## Preparing the environment

First, we should create a python virtual environment and install the required dependencies. To do so, we can run the following commands:

```bash
python -m venv data-quality
```
Now depending on your OS, you should run the following command:

- Linux/MacOS

```bash
source data-quality/bin/activate
```
- Windows

```PowerShell
data-quality\Scripts\Activate.ps1
```

Finally, we can install the required dependencies:

```bash
pip install -r requirements.txt
```

XXX Fix colab XXX

You can also launch the notebook in colab, clicking the following button: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## Basic Exploratory Data Analysis

First step in our analysis is to perform a basic exploratory data analysis. One simple library that provides useful features is SweetViz. Let's see how it works:

In [ ]:
# import pandas and sweetviz

import pandas as pd
import sweetviz as sv
import os


# load the dataset into a dataframe

bikes_file = os.path.join('data', 'input', 'bikes','bicycle-thefts - 4326.csv')
bikes_df = pd.read_csv(bikes_file)
bikes_df.head()

In [ ]:
# Basic pandas exploration
bikes_df.info()

In [ ]:
# Perform EDA using sweetviz

bikes_report = sv.analyze(bikes_df)
bikes_report.show_html('bikes_report.html')

## Review dataset information

There are some columns that look quite broken, like the missing DATES. Let's expand our search to other documents.

To explore the domain of this data, we can visit the following links:

- [Explore Bicycle Thefts Open Data](https://data.torontopolice.on.ca/datasets/TorontoPS::bicycle-thefts-open-data/explore)
- [About Bicycle Thefts Open Data](https://data.torontopolice.on.ca/datasets/TorontoPS::bicycle-thefts-open-data/about)
- [Public Safety Open Data Documentation](https://ago-item-storage.s3.amazonaws.com/c0b17f1888544078bf650f3b8b04d35d/PSDP_Open_Data_Documentation.pdf)
- [Bicycle Data Code Sheet](https://ago-item-storage.s3.amazonaws.com/332f6e958b704d9aa023e7c3487f710f/Bicycle_Data_Code_Sheet.pdf)

Things we can learn from the documentation:

"In accordance with the Municipal Freedom of Information and Protection of Privacy Act, the Toronto Police Service has taken the necessary measures to protect the privacy of individuals involved in the reported occurrences. **No personal information related to any of the parties involved in the occurrence will be released as open data.**"

The downloadable datasets display the REPORT_DATE and OCC_DATE fields in UTC timezone.

Latest update: 2023-10-07

Latest Open Data Portal update: 2023-11-21

## Defining our quality requirements

From the dataset description we can derive the following information:

- Location has not been able to be verified -> Coordinates may be blank or outside City of Toronto
- Fields have been included for both the old 140 City of Toronto Neighbourhoods structure as well as the new 158 City of Toronto Neighbourhoods structure
- This dataset contains Bicycle Thefts occurrences from 2014-2020.
- The location of crime occurrences have been deliberately offset to the nearest road intersection node to protect the privacy of parties involved in the occurrence. Due to the offset of occurrence location, the numbers by Division and Neighbourhood may not reflect the exact count of occurrences reported within these geographies
- Toronto Police Service does not guarantee the accuracy, completeness, timeliness of the data and it should not be compared to any other source of crime data
- There is no personal data included in this dataset, so we don't have to be extra careful with PII data.

In the Public Safety site, they claim the dataset covers bike thefts from 2014 to 2022, instead of 2020.

There is a site with data visualizations! https://data.torontopolice.on.ca/pages/bicycle-thefts

According to the Bicycle Thefts Dashboard: "Statistics are updated annually using the prepared open dataset provided for download. Current Year-to-Date data is not available for download." But... Current Year-to-Date data is visible in the Dashbord!

Let's explore data a bit:

- Historical data is available from 2014 to 2022. Yet, if we check the total numbers in the dashboard, the number of total thelfts is 31.972. In the dataset we have downloaded, we have 34.290 registered thefts. There is a difference of 2.318 registers (around 7% of the total)

There is another map, that covers all crime: https://torontops.maps.arcgis.com/apps/webappviewer/index.html?id=300d35778c114ef49d59454225043681

The documentation reports the following fields:

| Field | Field Name           | Description |
|-------|----------------------|-------------|
| 1     | EVENT_UNIQUE_ID      | Offence Number |
| 2     | PRIMARY_OFFENCE      | Primary Offence Type |
| 3     | OCC_DATE             | Date Offence Occurred (time is displayed in UTC format when downloaded as a CSV) |
| 4     | OCC_YEAR             | Year Offence Occurred |
| 5     | OCC_MONTH            | Month Offence Occurred |
| 6     | OCC_DOW              | Day of the Week Offence Occurred |
| 7     | OCC_DAY              | Day of the Month Offence Occurred |
| 8     | OCC_DOY              | Day of the Year Offence Occurred |
| 9     | OCC_HOUR             | Hour Offence Occurred |
| 10    | REPORT_DATE          | Date Offence was Reported (time is displayed in UTC format when downloaded as a CSV) |
| 11    | REPORT_YEAR          | Year Offence was Reported |
| 12    | REPORT_MONTH         | Month Offence was Reported |
| 13    | REPORT_DOW           | Day of the Week Offence was Reported |
| 14    | REPORT_DAY           | Day of the Month Offence was Reported |
| 15    | REPORT_DOY           | Day of the Year Offence was Reported |
| 16    | REPORT_HOUR          | Hour Offence was Reported |
| 17    | DIVISION             | Police Division where Offence Occurred |
| 18    | LOCATION_TYPE        | Location Type of Offence |
| 19    | PREMISES_TYPE        | Premises Type of Offence |
| 20    | BIKE_MAKE            | Make of Bicycle |
| 21    | BIKE_MODEL           | Model of Bicycle |
| 22    | BIKE_TYPE            | Type of Bicycle |
| 23    | BIKE_SPEED           | Speed of Bicycle |
| 24    | BIKE_COLOUR          | Colour of Bicycle |
| 25    | BIKE_COST            | Cost of Bicycle |
| 26    | STATUS               | Status of Bicycle |
| 27    | HOOD_158             | Identifier of Neighbourhood using City of Toronto's new 158 neighbourhood structure |
| 28    | NEIGHBOURHOOD_158    | Name of Neighbourhood using City of Toronto's new 158 neighbourhood structure |
| 29    | HOOD_140             | Identifier of Neighbourhood using City of Toronto's old 140 neighbourhood structure |
| 30    | NEIGHBOURHOOD_140    | Name of Neighbourhood using City of Toronto's old 140 neighbourhood structure |
| 31    | LONG_WGS84           | Longitude Coordinates (Offset to nearest intersection) |


Our dataset presents the following columns:

| Number | Column           | Description |
|--------|------------------|-------------|
| 0      | _id              | Unique row identifier for Open Data database |
| 1      | EVENT_UNIQUE_ID  | Offence Number |
| 2      | PRIMARY_OFFENCE  | Primary Offence Type |
| 3      | OCC_DATE         | Date of Offence |
| 4      | OCC_YEAR         | Year Offence Occurred |
| 5      | OCC_MONTH        | Month Offence Occurred |
| 6      | OCC_DOW          | Day of the Month Offence Occurred |
| 7      | OCC_DAY          | Day of the Year Offence Occurred |
| 8      | OCC_DOY          | Day of the Week Offence Occurred |
| 9      | OCC_HOUR         | Hour Offence Occurred |
| 10     | REPORT_DATE      | Date Offence was Reported |
| 11     | REPORT_YEAR      | Year Offence was Reported |
| 12     | REPORT_MONTH     | Month Offence was Reported |
| 13     | REPORT_DOW       | Day of the Month Offence was Reported |
| 14     | REPORT_DAY       | Day of the Year Offence was Reported |
| 15     | REPORT_DOY       | Day of the Week Offence was Reported |
| 16     | REPORT_HOUR      | Hour Offence was Reported |
| 17     | DIVISION         | Police Division where Offence Occurred |
| 18     | LOCATION_TYPE    | Location Type of Offence |
| 19     | PREMISES_TYPE    | Premises Type of Offence |
| 20     | BIKE_MAKE        | Make of Bicycle |
| 21     | BIKE_MODEL       | Model of Bicycle |
| 22     | BIKE_TYPE        | Type of Bicycle |
| 23     | BIKE_SPEED       | Speed of Bicycle |
| 24     | BIKE_COLOUR      | Colour of Bicycle |
| 25     | BIKE_COST        | Cost of Bicycle |
| 26     | STATUS           | Status of Bicycle |
| 27     | geometry         |             |

What are the differences between those tables? Let's check it out (helped by GPT-4):

| Field | Original Description                                    | New Description                                   | Difference |
|-------|---------------------------------------------------------|---------------------------------------------------|------------|
| 0     | *Not Present*                                           | Unique row identifier for Open Data database     | Added in new data |
| 1     | Offence Number                                          | Offence Number                                    | Same |
| 2     | Primary Offence Type                                    | Primary Offence Type                              | Same |
| 3     | Date Offence Occurred (UTC format in CSV)               | Date of Offence                                   | Description detail |
| 4     | Year Offence Occurred                                   | Year Offence Occurred                             | Same |
| 5     | Month Offence Occurred                                  | Month Offence Occurred                            | Same |
| 6     | Day of the Week Offence Occurred                        | Day of the Month Offence Occurred                 | Different |
| 7     | Day of the Month Offence Occurred                       | Day of the Year Offence Occurred                  | Different |
| 8     | Day of the Year Offence Occurred                        | Day of the Week Offence Occurred                  | Different |
| 9     | Hour Offence Occurred                                   | Hour Offence Occurred                             | Same |
| ...   | ...                                                     | ...                                               | ... |
| 27    | Identifier of Neighbourhood using City of Toronto's...  | *Not Present*                                     | Removed in new data |
| 28    | Name of Neighbourhood using City of Toronto's...        | *Not Present*                                     | Removed in new data |
| 29    | Identifier of Neighbourhood using City of Toronto's...  | *Not Present*                                     | Removed in new data |
| 30    | Name of Neighbourhood using City of Toronto's...        | *Not Present*                                     | Removed in new data |
| 31    | Longitude Coordinates (Offset to nearest intersection)  | *Not Present*                                     | Removed in new data |
| 27    | *Not Present*                                           | Geometry                                          | Added in new data |

Yes, the original column descriptions in the dataset are flipped out!

![Original Data Features](images\data_features.png)

But there is more... there is another dataset, more complete dataset in the Public Safety site!

### Let's explore the alternative dataset!

There is no Neighbourhood information in the Open Data site dataset! The dates are mostrly missing and from medieval times! Let's see if both issues are fixed in the alternative dataset.

In [ ]:
# Let's repeat the process for the Bicycle_Thefts_Open_Data_Public_Safety_version.csv

bikes_ps_file = os.path.join('data','input','bikes','Bicycle_Thefts_Open_Data_Public_Safety_version.csv')
bikes_ps_df = pd.read_csv(bikes_ps_file)
bikes_ps_df.head()

The Neighboorhood information is back! The dates look correct in ISO 8601 format but... we have got new colums X, Y with another coordinates! These do not appear in the documentation (But happen to be Web Mercator coordinates)

In [ ]:
# Let's create the sweetviz report for this new dataframe

bikes_ps_report = sv.analyze(bikes_ps_df)
bikes_ps_report.show_html('bikes_ps_report.html')


Observations:

There are almost no missing values in this dataset. Compare this to the original dataset!

The data that is not meant to be in the dataset (current year - 2023) is in the dataset, with 2257 entries. There are thefts reported from 1975 and 1983! Also, there about 50 entries prior to 2014 (which is the year the dataset is supposed to start).

Most of thefts are reported in OCC_DATE at 04:00:00. Maybe this is the time the pubs close? Are most of these happen during summer months? (Maybe this is after special events: https://bartenderatlas.com/features/travel/how-to-drink-in-toronto/)

Division has an NSA value. It is not documented in the guide.

Bike Make has 1% missing, but there are UNKNOWN MAKE and UNKNOWN in the dataset. Also, there is a mix of brands and shortcodes. This might make hard to analyze the data.

Bike Model has 38% of the data missing. There are UNKNOWN values in the dataset plus some brands used as model.

### What do we want to focus on now?

In the new file:

- Why do we have duplicated Event_Unique_ID? (There are 2.257 duplicated values)
- OCC_DATES always show 04:00:00 as time, but contradicts the OCC_HOUR column. We might need to fix the OCC_DATE
- Let's make sure we have the correct values for Division (Divisions should start with the letter D or be NSA)
- Let's replace the Bike Brand shortcodes with the brand name
- Let's check if the years includded in the dataset description match the years in the dataset
- Let's check if the dates are in the correct format

Extra
- Fix the date in our original example. Date columns should have dates.

In [ ]:
# Visualize the ooc_hours column using matplotlib

import matplotlib.pyplot as plt

plt.hist(bikes_ps_df['OCC_HOUR'], bins=24)

In [ ]:
# Display the rows that have EVENT_UNIQUE_ID = GO-20201550944

bikes_ps_df[bikes_ps_df['EVENT_UNIQUE_ID'] == 'GO-20201550944']

In [ ]:
dupe_bikes = bikes_ps_df[bikes_ps_df['EVENT_UNIQUE_ID'] == 'GO-20201550944']
dupe_bikes

## Converting our requirements into Great Expectations

We could start by visiting the GX gallery. But you must not do that! First, we need to plan and define our requirements. Let's do it!

### General Data Quality cycle

PDCA (Plan -> Do -> Check -> Act)

#### Plan

Let's review our observations:

- Event_Unique_ID has duplicate values (There are 2.257 duplicated values)
- OCC_DATE: most times show 04:00:00 as hour, but contradicts the OCC_HOUR column. 
- Districts: should be either NSA or starting by D
- Bike Brand should be either shortcodes or brand name, not a mixture of both
- Years should be in the range 2014-2023
- Dates should be in the correct format

Now let's sort them out based on inherent data quality dimensions: Accuracy, Completeness, Consistency, Credibility, Currentness.

- Accuracy: Dates should be in the correct format, bike brands should be either shortcodes or brand name, not a mixture of both
- Completeness: Years should be in the range 2014-2023
- Consistency: Districts should be either NSA or starting by D
- Currentness: Years should be in the range 2014-2023
- Credibility: no issues

Time to go back to slides before browsing the expectations gallery!


In [ ]:
# Bootstrapping the GX environment using fluent datasources

import great_expectations as gx

data_folder = os.path.join('data','input', 'bikes')

# Setting up a data context

context = gx.get_context()

datasource_name = "toronto_bike_thefts"
path_to_folder_containing_csv_files = data_folder

datasource = context.sources.add_pandas_filesystem(
    name=datasource_name, base_directory=path_to_folder_containing_csv_files
)



In [ ]:
# Creating a data asset from the datasource (folder might contain one or multiple files)

bike_thefts_data_asset = datasource.add_csv_asset(
    name = "original_bike_thefts_2014_2023",
    batching_regex = "bicycle-thefts - 4326\.csv",
)

In [ ]:
# Generate a batch request to recover all the contents of the given file

bike_thefts_batch_request = bike_thefts_data_asset.build_batch_request()

In [ ]:
# Generate an expectation suite

expectation_suite_name = "bike_thefts_expectations"

expectation_suite = context.add_or_update_expectation_suite(
    expectation_suite_name = expectation_suite_name
)

In [ ]:
# We create a validator, that allows us to interactively compare the data in the batch with the expectations in the suite

validator = context.get_validator(
    batch_request=bike_thefts_batch_request,
    expectation_suite_name=expectation_suite_name,
)

In [ ]:
# Let's convert our assertions into expectations
# Event_Unique_ID should have no duplicate values
# https://greatexpectations.io/expectations/expect_column_values_to_be_unique?filterType=Backend%20support&gotoPage=1&showFilters=true&viewType=Summary

validator.expect_column_values_to_be_unique(column='EVENT_UNIQUE_ID')

In [ ]:
# When we explore using a validator, we can update the expectations. If we save the expectations to the expectation suite, the last version for each element will be saved.

validator.expect_column_values_to_be_unique(column='EVENT_UNIQUE_ID', mostly=0.2 )

In [ ]:
# Let's check if the values in the DIVISION column start with a D
validator.expect_column_values_to_match_regex(column='DIVISION',regex='^D.*', meta={"Characteristic": "Accuracy", "QM": "Syntactic Accuracy"} )

In [ ]:
# District names should be either the values NSA or start by D

validator.expect_column_values_to_match_regex(column='DIVISION',regex='NSA|^D.*', meta={"Characteristic": "Accuracy", "QM": "Syntactic Accuracy"} )

In [ ]:
# Other alternative regexps
# Regex to match 'D' followed by any numerical characters: ^D\d+
# Regex that matches 'D' followed by 1 or 2 numbers: ^D\d{1,2}


In [ ]:
# Complete bike maker names must be incluided in the column BIKE_MAKE

# We can extract the list from the Bicycle Code Sheet

bike_makers = ['AQUILA',
 'ARGON 18',
 'BARLETTA',
 'BASSO',
 'BIANCHI',
 'BRODIE',
 'CANNONDALE',
 'CARRERA',
 'CCM',
 'CERVELO',
 'COLNAGO',
 'CUBE',
 'DEVINCI',
 'DIAMONDBACK',
 'ELECTRA',
 'EMMO',
 'FELT',
 'FUJI',
 'GARY FISHER',
 'GIANT',
 'GT',
 'HARO',
 'HUFFY',
 'HUSKY',
 'INFINITY',
 'IRON HORSE',
 'JAMIS',
 'KHS',
 'KLEIN',
 'KONA',
 'LAPIERRE',
 'LEMOND',
 'LITE SPEED',
 'LOUIS GARNEAU',
 'MARIN OR MARINONI',
 'MERIDA',
 'MIELE',
 'MIKADO',
 'MILLENNIUM',
 'MONGOOSE',
 'NAKAMURA',
 'NEXT',
 'NORCO',
 'OPUS',
 'ORYX',
 'OTHER',
 'PEUGEOT',
 'PINARELLO',
 'PIVOT',
 'PROTOUR',
 'RALEIGH',
 'REDLINE',
 'ROCKY MOUNTAIN',
 'SANTA CRUZ',
 'SCHWINN',
 'SCOTT',
 'SPECIALIZED',
 'SPORTEK',
 'SUPERCYCLE',
 'SUN',
 'SURLY',
 'TECH TEAM',
 'THIN BLUE LINE',
 'TREK',
 'TRUE NORTH',
 'TURCO',
 'VAGABOND',
 'VELO SPORT',
 'YETI',
 'UNKNOWN MAKE']

In [ ]:
bike_dict = {'AQUI': 'AQUILA',
 'ARGO': 'ARGON 18',
 'BARL': 'BARLETTA',
 'BASS': 'BASSO',
 'BIAN': 'BIANCHI',
 'BROD': 'BRODIE',
 'CANN': 'CANNONDALE',
 'CARR': 'CARRERA',
 'CC': 'CCM',
 'CERV': 'CERVELO',
 'COLN': 'COLNAGO',
 'CUBE': 'CUBE',
 'DEVI': 'DEVINCI',
 'DIAM': 'DIAMONDBACK',
 'ELEC': 'ELECTRA',
 'EM': 'EMMO',
 'FELT': 'FELT',
 'FUJI': 'FUJI',
 'GARY': 'GARY FISHER',
 'GI': 'GIANT',
 'GT': 'GT',
 'HARO': 'HARO',
 'HUFF': 'HUFFY',
 'HUSK': 'HUSKY',
 'INFI': 'INFINITY',
 'IRON': 'IRON HORSE',
 'JAMI': 'JAMIS',
 'KHS': 'KHS',
 'KLEI': 'KLEIN',
 'KONA': 'KONA',
 'LAPI': 'LAPIERRE',
 'LEMO': 'LEMOND',
 'LITE': 'LITE SPEED',
 'LOUI': 'LOUIS GARNEAU',
 'MARI': 'MARIN OR MARINONI',
 'MERI': 'MERIDA',
 'MIEL': 'MIELE',
 'MIKA': 'MIKADO',
 'MILL': 'MILLENNIUM',
 'MONG': 'MONGOOSE',
 'NAKA': 'NAKAMURA',
 'NEXT': 'NEXT',
 'NO': 'NORCO',
 'OPUS': 'OPUS',
 'ORYX': 'ORYX',
 'OT': 'OTHER',
 'PEUG': 'PEUGEOT',
 'PINA': 'PINARELLO',
 'PIVO': 'PIVOT',
 'PROT': 'PROTOUR',
 'RA': 'RALEIGH',
 'REDL': 'REDLINE',
 'ROCK': 'ROCKY MOUNTAIN',
 'SANT': 'SANTA CRUZ',
 'SC': 'SCHWINN',
 'SCOT': 'SCOTT',
 'SE': 'SPECIALIZED',
 'SPOR': 'SPORTEK',
 'SU': 'SUPERCYCLE',
 'SUN': 'SUN',
 'SURL': 'SURLY',
 'TECH': 'TECH TEAM',
 'THIN': 'THIN BLUE LINE',
 'TR': 'TREK',
 'TRUE': 'TRUE NORTH',
 'TURC': 'TURCO',
 'VAGA': 'VAGABOND',
 'VELO': 'VELO SPORT',
 'YETI': 'YETI',
 'UNKNOWN': 'UNKNOWN MAKE'}

In [ ]:
# Now we will check the contents of BIKE_MAKE against our list of complete bike maker names using the expectation expect_column_values_to_be_in_set

validator.expect_column_values_to_be_in_set(column='BIKE_MAKE', value_set=bike_makers )

In [ ]:
# Now we will check the contents of BIKE_MAKE against the short codes using the expectation expect_column_values_to_be_in_set and the keys in the bike_dict

bike_codes = list(bike_dict.keys())

validator.expect_column_values_to_be_in_set(column='BIKE_MAKE', value_set=bike_codes, meta={"Characteristic": "XXX", "QM": "XXX"} )

In [ ]:
# Now let's detect values that are not in the code sheet either as codes or brands

merged_list = bike_makers + bike_codes

# We eliminate duplicate values
merged_list_unique = list(set(merged_list))

validator.expect_column_values_to_be_in_set(column='BIKE_MAKE', value_set=merged_list )

In [ ]:
# All dates should be in the correct format

validator.expect_column_values_to_be_dateutil_parseable(column='OCC_DATE', meta={"Characteristic": "Accuracy", "QM": "Syntactic Accuracy"} )

In [ ]:
validator.expect_column_values_to_be_dateutil_parseable(column='REPORT_DATE', meta={"Characteristic": "Accuracy", "QM": "Syntactic Accuracy"} )

In [ ]:
# Expecting that years should be in the range 2014-2023
validator.expect_column_values_to_be_between(column='OCC_YEAR', min_value=2014, max_value=2023, meta={"Characteristic": "Completeness"} )

In [ ]:
# Let's be a little bit less strict and allow for 2013 as well
validator.expect_column_values_to_be_between(column='OCC_YEAR', min_value=2013, max_value=2023 )

In [ ]:
# Let's check the REPORT_YEAR column now

validator.expect_column_values_to_be_between(column='REPORT_YEAR', min_value=2013, max_value=2023)

So someone reported in 2023 that their bike was stolen in 1975! In all previous descriptions, about dates, we should change dates by reported dates. It doesn't matter when the bike was stolen, but when it was reported.

In [ ]:
# Let's make sure that no empty values are allowed in the OCC_DATE or REPORT_DATE columns

validator.expect_column_values_to_not_be_null(column='OCC_DATE')
validator.expect_column_values_to_not_be_null(column='REPORT_DATE')



Can you find other alternative expectations that could be tested against dates? (Hint: Visit the gallery and report back)

In [ ]:
# Now we create a checkpoint to store all the data related to expectation validation

checkpoint_pre_fixes = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_bike_theft_pre_fixes",
    data_context=context,
    validations=[
        {
            "batch_request": bike_thefts_batch_request,
            "expectation_suite_name": expectation_suite_name,
        },
    ],
)

In [ ]:
# Now we save our expectations
validator.save_expectation_suite(discard_failed_expectations=False)

In [ ]:
# Now we run the checkpoint to validate the expectations against the data we extracted from the batch_request

checkpoint_result = checkpoint_pre_fixes.run()

In [ ]:
# And last, but not least, we generate a report using the DataDocs feature from Great Expectations

context.build_data_docs()

## Fixing problems detected using pandas

IMHO Raw data should not be mutable, otherwise we lose track of the changes, the tests, etc. So first step, let's create a copy of the original dataset in the input folder into the interim folder.

 

In [ ]:
# Let's create a copy of the original dataset in the input folder into the interim folder

import shutil

shutil.copyfile(os.path.join('data','input', 'bikes', 'bicycle-thefts - 4326.csv'), os.path.join('data','interim', 'bikes', 'bicycle-thefts - 4326.csv'))

# Note: We will be working with another dataframe. THIS DOES NOT MODIFY ANY THE DATE WE HAVE ALREADY VALIDATED 

bike_thefts_df = pd.read_csv(os.path.join('data', 'interim', 'bikes', 'bicycle-thefts - 4326.csv'))

We will perform all the changes in the interim folder, modifying the dataset in place. We will perform four fixes and a half (fictional) fix! XXX 

In [ ]:
# Get the 25 most common values in the BIKE_MAKE column

bike_thefts_df['BIKE_MAKE'].value_counts().head(25)

## FIX1: Fixing (partially) the bike brands

In [ ]:
# Fix 1 - Replace the bike codes with the bike make (longer form is more explicit) using the bike_dict

bike_thefts_df['BIKE_MAKE'] = bike_thefts_df['BIKE_MAKE'].replace(bike_dict)

In [ ]:
bike_thefts_df['BIKE_MAKE'].value_counts().head(25)

Depending of our goal we could perform additional fixes. For example, we could replace all missing values with UNKNOWN MAKE, so we have no missing values. We could also replace all the failing rows with the OTHER brand, as thoese are not tracked in the report. This would make us to lose some details about the brands involved, so it's not such a good idea.

The best idea: report back to the source that the Bicycle Code Sheet is incomplete, as it doesn't contain all codified values. Also, ask for a review of the data, as there are brands that look like BIKE_TYPE instead of make like E-BIKE.

In [ ]:
# Let's replace missing values with the string 'UNKNOWN MAKE'

bike_thefts_df['BIKE_MAKE'] = bike_thefts_df['BIKE_MAKE'].fillna('UNKNOWN MAKE')

## FIX 2: Fixing the dates

In [ ]:
# Fix 2 - Let's recreate the OCC_DATE column using the OCC_YEAR, OCC_MONTH and OCC_DAY, OCC_HOUR columns, producing an ISO 8601 compliant date

bike_thefts_df["OCC_DATE"] = (
    bike_thefts_df["OCC_YEAR"].astype(str)
    + "-"
    + bike_thefts_df["OCC_MONTH"].astype(str).str.zfill(2)
    + "-"
    + bike_thefts_df["OCC_DAY"].astype(str).str.zfill(2)
    + "T"
    + bike_thefts_df["OCC_HOUR"].astype(str).str.zfill(2)
    + ":00:00"
)

In [ ]:
bike_thefts_df['OCC_DATE'].sample(10)

Checking the data, we discover that OCC_MONTH has month names instead of month numbers, so we are not able to perform the previous operation. Let's fix it!

In [ ]:
# We need to convert the month names into the corresponding month number

month_dict = {
    '01': 'January',
    '02': 'February',
    '03': 'March',
    '04': 'April',
    '05': 'May',
    '06': 'June',
    '07': 'July',
    '08': 'August',
    '09': 'September',
    '10': 'October',
    '11': 'November',
    '12': 'December'
}

In [ ]:
# We inverse the dictionary 
month_number_dict = {v: k for k, v in month_dict.items()}

bike_thefts_df['OCC_MONTH'] = bike_thefts_df['OCC_MONTH'].map(month_number_dict)


In [ ]:
bike_thefts_df["OCC_DATE"] = (
    bike_thefts_df["OCC_YEAR"].astype(str)
    + "-"
    + bike_thefts_df["OCC_MONTH"].astype(str).str.zfill(2)
    + "-"
    + bike_thefts_df["OCC_DAY"].astype(str).str.zfill(2)
    + "T"
    + bike_thefts_df["OCC_HOUR"].astype(str).str.zfill(2)
    + ":00:00"
)

In [ ]:
bike_thefts_df['OCC_DATE'] = pd.to_datetime(bike_thefts_df['OCC_DATE'])
bike_thefts_df['OCC_DATE'].sample(10)

In [ ]:
# We will repeat the process for the REPORT_DATE column

bike_thefts_df["REPORT_MONTH"] = bike_thefts_df["REPORT_MONTH"].map(month_number_dict)

bike_thefts_df["REPORT_DATE"] = (
    bike_thefts_df["REPORT_YEAR"].astype(str)
    + "-"
    + bike_thefts_df["REPORT_MONTH"].astype(str).str.zfill(2)
    + "-"
    + bike_thefts_df["REPORT_DAY"].astype(str).str.zfill(2)
    + "T"
    + bike_thefts_df["REPORT_HOUR"].astype(str).str.zfill(2)
    + ":00:00"
)

bike_thefts_df['REPORT_DATE'] = pd.to_datetime(bike_thefts_df['REPORT_DATE'])

bike_thefts_df['REPORT_DATE'].sample(10)

## Additional fixes

Those were our fixes for the original file, that had some missing dates. But it has no neighbourhood information. So let's now focus on fixing the alternative file too.

In [ ]:
# Let's create a copy of the original alternate dataset and load it as dataframe

shutil.copyfile(os.path.join('data','input', 'bikes', 'Bicycle_Thefts_Open_Data_Public_Safety_version.csv'), os.path.join('data','interim', 'bikes', 'Bicycle_Thefts_Open_Data_Public_Safety_version.csv'))

bike_ps_df = pd.read_csv(os.path.join('data', 'interim', 'bikes', 'Bicycle_Thefts_Open_Data_Public_Safety_version.csv'))

bike_ps_df.head()

In [ ]:
# Extra Fix 3 - Spiltting the compound columns into separate columns (NEIGHBOURHOOD_140 column includes the neighbourhood name and the neighbourhood code into the same column
# The code is available also as a separate column HOOD_140, so we need to remove it from the column. We will do this using a regex to remove the (nn) part at the end of the string

import re

def remove_code(text):
    return re.sub(r'\s\(\d+\)', '', text)

bike_ps_df['NEIGHBOURHOOD_140'] = bike_ps_df['NEIGHBOURHOOD_140'].apply(remove_code)

# The simpler did not work (I assume because of the datatype not being str! bike_ps_df['NEIGHBOURHOOD_140'] = bike_ps_df['NEIGHBOURHOOD_140'].str.replace(r'\s\(\d+\)', '')

bike_ps_df['NEIGHBOURHOOD_140'].head()


In a real scenario, we would have started with the alternative file, as it has more data. In this tutorial, we wanted to show how to fix data. But now, it's time to extract those columns and merge them with the original file. 

In [ ]:
# Create a subset of bike_ps_df with only the required columns
bike_ps_subset = bike_ps_df[['OBJECTID', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140']]

# Merge both dataframe on their primary indexes using a left join

merged_df = pd.merge(bike_thefts_df, bike_ps_subset, left_on='_id', right_on='OBJECTID', how='left')

# Drop the duplicate index

merged_df.drop(columns=['OBJECTID'], inplace=True)

# Check the results

merged_df.head()


In [ ]:
# Let's do a brief EDA of the merged dataframe using sweetviz again, focusing on the new columns

merged_report = sv.analyze(merged_df)
merged_report.show_html('merged_report.html')

Once we have done all the fixes, it's time to export it as a new file. We will export it to the interim folder, as it's a temporary file.

In [ ]:
# We will save this dataframe as bike_thefts_merged.csv in the interim folder

merged_df.to_csv(os.path.join('data', 'interim', 'bikes', 'bike_thefts_merged.csv'), index=False)

## Assesing data quality improvements after remediation steps were taken

In [ ]:
# Defining the environment for this new file

# Data Context: Done - Value: toronto_bike_thefts
# Expectation Suite: Done - Value: bike_thefts_expectations
# Data Source: TBD - toronto_bike_thefts_fixed (We are loading the data from the interim folder instead of the input folder. It is a different folder (and also includes many files))
# Data Asset: TBD - bike_thefts_fixed_v1 (We need to explicitely pass the file name bike_thefts_merged.csv)
# Batch Request: TBD - batch_request_bike_thefts_fixed_v1 - We will load all the registers from the new file)
# Checkpoint: TBD - checkpoint_bike_theft_post_fixes_v1

# Setting the data source

datasource_fixed_name = "toronto_bike_thefts_fixed"
path_to_folder_containing_fixed_csv_files = os.path.join('data','interim', 'bikes')

datasource_fixed = context.sources.add_pandas_filesystem(
    name=datasource_fixed_name, base_directory=path_to_folder_containing_fixed_csv_files
)

# Creating the data asset

bike_thefts_fixed_data_asset = datasource_fixed.add_csv_asset(
    name = "bike_thefts_fixed_v1",
    batching_regex = "bike_thefts_merged\.csv",
)

# Generate a batch request to recover all the contents of the given file

batch_request_bike_thefts_fixed_v1 = bike_thefts_fixed_data_asset.build_batch_request()

# Generate a checkpoint

checkpoint_post_fixes_v1 = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_bike_theft_post_fixes_v1",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request_bike_thefts_fixed_v1,
            "expectation_suite_name": expectation_suite_name,
        },
    ],
)

In [ ]:
# Now we run the checkpoint to validate the original expectation suite against the data we extracted from the new batch_request

checkpoint_post_fixes_v1_result = checkpoint_post_fixes_v1.run()

In [ ]:
# And last, but not least, we generate a report using the DataDocs feature from Great Expectations

context.build_data_docs()

## What have we learned?

Quality is a continuous process. Reports will help you keep track of the improvements but you might not get a 100% quality datasets. You can bend the expectations or modify the data, depending on your needs. Think also about what how do you plan to integrate it, visualize it, etc.